# Homework Week 2


### Q1. Runing Ollama with Docker
```bash
docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama 

docker exec -it ollama /bin/bash 
ollama --version
```

ollama version : ollama version is 0.1.48

### Q2. Downloading an LLM

```bash
ollama pull gemma:2b
```

In docker, it saved the results into `/root/.ollama`

We're interested in the metadata about this model. You can find
it in `models/manifests/registry.ollama.ai/library`

What's the content of the file related to gemma?

```json
{"schemaVersion":2,
"mediaType":"application/vnd.docker.distribution.manifest.v2+json",
"config":{"mediaType":"application/vnd.docker.container.image.v1+json",
            "digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290",
            "size":483},
"layers":[{"mediaType":"application/vnd.ollama.image.model",
            "digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12",
            "size":1678447520},
        {"mediaType":"application/vnd.ollama.image.license",
        "digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca",
        "size":8433},
        {"mediaType":"application/vnd.ollama.image.template",
        "digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871",
        "size":136},
        {"mediaType":"application/vnd.ollama.image.params",
        "digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0",
        "size":84}]} 
```

### Q3. Running the LLM

Test the following prompt: "10 * 10". What's the answer?

In [14]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [15]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [16]:
res = llm('10 * 10')
print(res)

Sure, here is the answer:

10 * 10 = 100




## Q4. Donwloading the weights 

We don't want to pull the weights every time we run
a docker container. Let's do it once and have them available
every time we start a container.

First, we will need to change how we run the container.

Instead of mapping the `/root/.ollama` folder to a named volume,
let's map it to a local directory:

```bash
mkdir ollama_files

docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

Now pull the model:

```bash
docker exec -it ollama ollama pull gemma:2b 
```

What's the size of the `ollama_files/models` folder? 

``` bash
du -sh ollama_files/models/
1.6G    ollama_files/models/
```

### Q5. Adding the weights
```bash
# Use the base Ollama image from Docker Hub
FROM ollama/ollama

# Copy model weights from local directory to Docker image
COPY ./ollama_files/models /root/.ollama/models
```

### Q6. Serving it

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    return response.choices[0].message.content

In [26]:
prompt = "What's the formula for energy?"

res = llm(prompt)
print(res)

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2mv^2**

**Potential energy (U)** is the energy an object possesses due to its position or configuration. It is calculated as the product of an object's mass, gravitational constant (g), and height or position above a reference point.

**U = mgh**

Where:

* **m** is the mass in kilograms (kg)
* **g** is the gravitational constant (9.8 m/s^2)
* **h** is the height or position in meters (m)

The formula shows that energy can be expressed as the sum of kinetic and potential energy. The kinetic energy is a measure of the object's ability to do work, while the potential energy is a measure of the object's ability to do w

In [28]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")


# Tokenize the results
tokens = encoding.encode(res)

# Calculate the number of tokens
num_tokens = len(tokens)

print(f"Number of tokens: {num_tokens}")

Number of tokens: 261
